In [1]:
pip install iocextract

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials


from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import iocextract
import urllib.parse
import pandas as pd
import csv
import re
import ast
import requests
import urllib.request
from urllib.request import urlopen


In [4]:

#function to search CVE
def searchCVE(regular_expression,content_text):
    result=[]
    result=re.findall(regular_expression,content_text,re.IGNORECASE)
    return result

In [5]:

#function to search Domain
def searchDomain(regular_expression,content_text):
    result=[]
    result=re.findall(regular_expression,content_text,re.IGNORECASE)
    return result

In [6]:
#function to create IOC List
def saveIOC(row,list_ioc_data,typeioc):
  if len(list_ioc_data[row][typeioc])!=0:
    for j in range(0,len(list_ioc_data[row][typeioc])):
      IOC_Final={
						'tweet_date':list_ioc_data[row]['Created At'],
						'account':list_ioc_data[row]['User Name'],
						'ioc_type':typeioc,
						'ioc_value':list_ioc_data[row][typeioc][j],
						'type_of_attack':list_ioc_data[row]['Search Hashtag'],
						'tweet_id':list_ioc_data[row]['Tweet ID'],
						'text': list_ioc_data[row]['Text'],
						'Mentioned Hashtag':list_ioc_data[row]['Mentioned Hashtag'],
						'Expanded URL':list_ioc_data[row]['Expanded URL'],
						
						}
      IOC_Final_list.append(IOC_Final)

In [ ]:
tweets = pd.read_csv("/content/drive/MyDrive/output/relevant_tweet.csv",lineterminator='\n')
list_ioc_data= []
IOC_Final_list=[]
cves=[]
domains=[]
hashtag_list=['malware','ransomware','botnet','androidtrojan','bankingtrojan','windowstrojan','remoteaccesstrojan','rattrojan','linuxtrojan','adware','spyware','keylogger','rootkit','bootkit','rigek','exploitkit','honeypot','ioc','malspam','crypto-locker','cryptolocker','crypto-ransomware','opendir','c2','md5','sha256','spearphishing','spear-phishing','phishing','phishingkit','cyberattack','cve','rootkit','sqlinjection','keylogging','0day','keylogger','passwordcracker','metasploit','reduc','zombie','C&C']
#domain_expression_old="(?<=\.)([^.]+)(?:\.(?:co\.uk|ac\.us|com|net|org|info|coop|int|co|ac|ie|co|ai|eu|ca|icu|top|xyz|tk|cn|ga|cf|nl|us|eu|de|hk|am|tv|bingo|blackfriday|gov|edu|mil|arpa|au|ru|[^.]+(?:$|\n)))"
domain_expression = r'(?:[a-zA-Z0-9](?:[a-zA-Z0-9\-]{,61}[a-zA-Z0-9])?\.)+[a-zA-Z]{2,6}'
cve_expression="CVE-\d{4}-\d{4,7}"
youtube_expression="(?:https?:\/\/)?(?:www\.)?youtu\.?be(?:\.com)?\/?.*(?:watch|embed)?(?:.*v=|v\/|\/)([\w\-_]+)\&?"
twitter_expression="(?:http?:\/\/)?(?:www\.)?twitter\.com\/(?:(?:\w)*#!\/)?(?:pages\/)?(?:[\w\-]*\/)*([\w\-]*)"
facebook_expression="(?:https?:\/\/)?(?:www\.)?(mbasic.facebook|m\.facebook|facebook|fb)\.(com|me)\/(?:(?:\w\.)*#!\/)?(?:pages\/)?(?:[\w\-\.]*\/)*([\w\-\.]*)"

In [8]:
tweets

,Unnamed: 0,Unnamed: 0.1,Author ID,User Name,Screen Name,Tweet ID,Created At,Text,Like Count,Quote Count,...,ID of Referenced Tweets,Type of Referenced Tweets,Attachments,Location tagged by the user,Language,Sensitive URL,Reply Settings,Source,preprocess_text,isValid
0,2414640,59024,2.401309e+07,MalwarePatrol,Malware Patrol,1.344798e+18,2021-01-01 00:09:03+00:00,Another active #Phishing targeting Google Driv...,1.0,0.0,...,NaN,NaN,{'media_keys': ['3_1344797749137600514']},NaN,en,False,everyone,MP_Post_New_Malware_Information,another active phishing target google drive ur...,1
1,2414566,58950,3.973558e+09,PhishStats,Phish.Stats 🐟,1.344802e+18,2021-01-01 00:26:26+00:00,new #phishing at hXXp://departmen-advant-page[...,2.0,1.0,...,NaN,NaN,{'media_keys': ['3_1344802123939278852']},NaN,en,False,everyone,PhishStats,new phishing [defanged] [ip] united state alch...,1
2,2414553,58937,1.944418e+07,ecarlesi,Emiliano Carlesi,1.344803e+18,2021-01-01 00:31:00+00:00,Possible threat on hxxps://teamfrenette[.]com/...,0.0,0.0,...,NaN,NaN,NaN,NaN,en,False,everyone,PhishingPicker,possible threat [defanged] phishing opendir,1
3,2414545,58929,1.944418e+07,ecarlesi,Emiliano Carlesi,1.344803e+18,2021-01-01 00:31:25+00:00,Possible threat on hxxps://teamfrenette[.]com/...,0.0,0.0,...,NaN,NaN,NaN,NaN,en,False,everyone,PhishingPicker,possible threat [defanged] phishing opendir,1
4,2414333,58717,1.944418e+07,ecarlesi,Emiliano Carlesi,1.344825e+18,2021-01-01 01:58:06+00:00,Threat on hxxp://fraudcheck-barclays[.]info/Ac...,0.0,0.0,...,NaN,NaN,NaN,NaN,en,False,everyone,PhishingPicker,threat [defanged] phishing,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58764,306,306,5.789760e+08,phishunt_io,phishunt.io,1.421606e+18,2021-07-31 22:56:45+00:00,#NewPhishing | #phishing #scam\n\n🌐 /bankofame...,1.0,0.0,...,NaN,NaN,{'media_keys': ['3_1421605781359583235']},NaN,en,False,everyone,phishunt,newphishing phishing scam bankofamericamiamico...,1
58765,264,264,2.504830e+09,BethkazV,Beth Kazakewich,1.421608e+18,2021-07-31 23:06:42+00:00,Test your knowledge at https://t.co/yvg8zciMOZ...,1.0,0.0,...,NaN,NaN,NaN,NaN,en,False,everyone,Twitter Web App,test knowledge eset antivirus phishing contest...,1
58766,234,234,1.207782e+18,Coug2Coug,Coug2CougPodcast,1.421610e+18,2021-07-31 23:14:20+00:00,@brookechesney @EasopWinston Go Cougs #CvE,0.0,0.0,...,1.421608e+18,replied_to,NaN,NaN,en,False,everyone,Twitter for iPhone,go cougs cve,1
58767,115,115,1.944418e+07,ecarlesi,Emiliano Carlesi,1.421616e+18,2021-07-31 23:37:40+00:00,Threat on hxxp://certiport[.]games/postale[.]z...,0.0,0.0,...,NaN,NaN,"{'media_keys': ['3_1421616066598522880', '3_14...",NaN,en,False,everyone,PhishingPicker,threat [defanged] phishing opendir namecom,1


In [9]:
for ind in range(0,len(tweets)):
	full_url=[]
	domain_list=[]
	if tweets['Language'][ind]=='en' and tweets['Type of Referenced Tweets'][ind]!='retweeted':
		content=tweets['Text'][ind]
		list_url=list(iocextract.extract_urls(content, refang=True))
		if isinstance(tweets['Expanded URL'][ind], str):
			expanded_url=eval(tweets['Expanded URL'][ind])
			if len(list_url)==len(expanded_url):
				for url_ind in range(0,len(list_url)):
					list_url[url_ind]=expanded_url[url_ind]
		full_url=list_url.copy()
		for i in range(0,len(full_url)):
			if(re.search(twitter_expression,full_url[i]) or re.search(youtube_expression,full_url[i]) or re.search(facebook_expression,full_url[i])):
				list_url.remove(full_url[i])
		ip=list(iocextract.extract_ips(content, refang=True))
		ipv4=list(iocextract.extract_ipv4s(content, refang=True))
		ipv6=list(iocextract.extract_ipv6s(content))
		total_ip=list(set(ip+ipv4+ipv6))
		length=len(total_ip)
		date_list=[]
		hashtags=[]
		lower_hashtag=[]
		if(length>0):
			for loc in range(0,length):
				ip_split=str(total_ip[loc]).split(":")
				if(len(ip_split)>1):
					date_list.append(total_ip[loc])
			for dateloc in range(0,len(date_list)):
				total_ip.remove(date_list[dateloc])
		hashes=list(iocextract.extract_hashes(content))
		"""for i in list_url:
			domains=searchDomain(domain_expression,i)
		domain_list=domains.copy()
		for i in range(0,len(domain_list)):
			 if(domain_list[i].isnumeric()):
				 domains.remove(domain_list[i])"""
		domains = searchDomain(domain_expression,content)
		cves=searchCVE(cve_expression,content)
		mails=list(iocextract.extract_emails(content,refang=True))
		list_yara_rules=list(iocextract.extract_yara_rules(content))
		t_hashtag=tweets['Hashtag'][ind]
		if(isinstance(t_hashtag, float)==False):
			t_hashtag=ast.literal_eval(tweets['Hashtag'][ind])
			for h in t_hashtag:
				 lower_hashtag.append(str(h).lower())
			for item in hashtag_list:
				if(item in lower_hashtag):
					hashtags.append(item)
			
	 		
		columns=[list_url,total_ip,hashes,domains,cves,mails,list_yara_rules]
		flag=0
		for i in columns:
			if i:
				flag=1
				break
		if flag==1:
			ioc_data = {
				'Created At':tweets['Created At'][ind],	
				'User Name':tweets['User Name'][ind],
				'Tweet ID':tweets['Tweet ID'][ind],
				'Text': content,
				'Mentioned Hashtag':tweets['Hashtag'][ind],
				'Search Hashtag':hashtags,
				'Expanded URL':tweets['Expanded URL'][ind],
				'url' : list_url,
				'ip' : total_ip, 
				'hash':hashes,
				'domain':domains,
				'CVE': cves,
				'email' : mails,
				'yara': list_yara_rules}
			list_ioc_data.append(ioc_data)

In [10]:
for row in range(0,len(list_ioc_data)):
      saveIOC(row,list_ioc_data,'url')
      saveIOC(row,list_ioc_data,'ip')
      saveIOC(row,list_ioc_data,'hash')
      saveIOC(row,list_ioc_data,'domain')
      saveIOC(row,list_ioc_data,'CVE')
      saveIOC(row,list_ioc_data,'email')
      saveIOC(row,list_ioc_data,'yara')


In [11]:
df1 = pd.DataFrame(list_ioc_data)
df2 = pd.DataFrame(IOC_Final_list)

In [12]:
df1

,Created At,User Name,Tweet ID,Text,Mentioned Hashtag,Search Hashtag,Expanded URL,url,ip,hash,domain,CVE,email,yara
0,2021-01-01 00:09:03+00:00,MalwarePatrol,1.344798e+18,Another active #Phishing targeting Google Driv...,"['Phishing', 'onpatrol4malware']",[phishing],['https://twitter.com/MalwarePatrol/status/134...,"[http://dubioustimes.com/, https://t.co/m4mjDM...",[],[],"[dubioustimes.com, t.co]",[],[],[]
1,2021-01-01 00:26:26+00:00,PhishStats,1.344802e+18,new #phishing at hXXp://departmen-advant-page[...,"['phishing', 'infosec', 'cybersecurity']",[phishing],['https://twitter.com/PhishStats/status/134480...,"[http://departmen-advant-page.my.id/, https://...",[205.144.171.153],[],[t.co],[],[],[]
2,2021-01-01 00:31:00+00:00,ecarlesi,1.344803e+18,Possible threat on hxxps://teamfrenette[.]com/...,"['phishing', 'opendir']","[opendir, phishing]",NaN,[http://teamfrenette.com/wp-content/plugins/el...,[],[],[],[],[],[]
3,2021-01-01 00:31:25+00:00,ecarlesi,1.344803e+18,Possible threat on hxxps://teamfrenette[.]com/...,"['phishing', 'opendir']","[opendir, phishing]",NaN,[http://teamfrenette.com/wp-content/plugins/el...,[],[],[],[],[],[]
4,2021-01-01 01:58:06+00:00,ecarlesi,1.344825e+18,Threat on hxxp://fraudcheck-barclays[.]info/Ac...,['phishing'],[phishing],NaN,"[http://fraudcheck-barclays.info/Activity/, ht...",[],[],[],[],[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58400,2021-07-31 22:55:02+00:00,HeliosCert,1.421605e+18,Sample submitted\n2021-07-31 22:55:02\nDionaea...,"['malware', 'cyber', 'security']",[malware],NaN,[],[178.238.78.124],[4436f92229f527236c0847564351ef5147b2458a120f3...,[],[],[],[]
58401,2021-07-31 22:56:45+00:00,phishunt_io,1.421606e+18,#NewPhishing | #phishing #scam\n\n🌐 /bankofame...,"['NewPhishing', 'phishing', 'scam']",[phishing],['https://twitter.com/phishunt_io/status/14216...,[],[217.160.0.240],[],"[bankofamerica-miami.com, t.co]",[],[],[]
58402,2021-07-31 23:06:42+00:00,BethkazV,1.421608e+18,Test your knowledge at https://t.co/yvg8zciMOZ...,"['eset', 'antivirus', 'phishing', 'contest', '...",[phishing],['https://www.esetphishingderby.com?score=65'],[https://www.esetphishingderby.com?score=65],[],[],[t.co],[],[],[]
58403,2021-07-31 23:37:40+00:00,ecarlesi,1.421616e+18,Threat on hxxp://certiport[.]games/postale[.]z...,"['phishing', 'opendir', 'namedotcom']","[opendir, phishing]",['https://twitter.com/ecarlesi/status/14216160...,[],[],[],[t.co],[],[],[]


In [13]:
df2

,tweet_date,account,ioc_type,ioc_value,type_of_attack,tweet_id,text,Mentioned Hashtag,Expanded URL
0,2021-01-01 00:09:03+00:00,MalwarePatrol,url,http://dubioustimes.com/,[phishing],1.344798e+18,Another active #Phishing targeting Google Driv...,"['Phishing', 'onpatrol4malware']",['https://twitter.com/MalwarePatrol/status/134...
1,2021-01-01 00:09:03+00:00,MalwarePatrol,url,https://t.co/m4mjDMt7az,[phishing],1.344798e+18,Another active #Phishing targeting Google Driv...,"['Phishing', 'onpatrol4malware']",['https://twitter.com/MalwarePatrol/status/134...
2,2021-01-01 00:09:03+00:00,MalwarePatrol,domain,dubioustimes.com,[phishing],1.344798e+18,Another active #Phishing targeting Google Driv...,"['Phishing', 'onpatrol4malware']",['https://twitter.com/MalwarePatrol/status/134...
3,2021-01-01 00:09:03+00:00,MalwarePatrol,domain,t.co,[phishing],1.344798e+18,Another active #Phishing targeting Google Driv...,"['Phishing', 'onpatrol4malware']",['https://twitter.com/MalwarePatrol/status/134...
4,2021-01-01 00:26:26+00:00,PhishStats,url,http://departmen-advant-page.my.id/,[phishing],1.344802e+18,new #phishing at hXXp://departmen-advant-page[...,"['phishing', 'infosec', 'cybersecurity']",['https://twitter.com/PhishStats/status/134480...
...,...,...,...,...,...,...,...,...,...
195509,2021-07-31 23:06:42+00:00,BethkazV,url,https://www.esetphishingderby.com?score=65,[phishing],1.421608e+18,Test your knowledge at https://t.co/yvg8zciMOZ...,"['eset', 'antivirus', 'phishing', 'contest', '...",['https://www.esetphishingderby.com?score=65']
195510,2021-07-31 23:06:42+00:00,BethkazV,domain,t.co,[phishing],1.421608e+18,Test your knowledge at https://t.co/yvg8zciMOZ...,"['eset', 'antivirus', 'phishing', 'contest', '...",['https://www.esetphishingderby.com?score=65']
195511,2021-07-31 23:37:40+00:00,ecarlesi,domain,t.co,"[opendir, phishing]",1.421616e+18,Threat on hxxp://certiport[.]games/postale[.]z...,"['phishing', 'opendir', 'namedotcom']",['https://twitter.com/ecarlesi/status/14216160...
195512,2021-07-31 23:50:02+00:00,HeliosCert,ip,185.27.116.4,[malware],1.421619e+18,Sample submitted\n2021-07-31 23:50:02\nDionaea...,"['malware', 'cyber', 'security']",NaN


In [ ]:
df1 = pd.DataFrame(list_ioc_data)
df1.to_csv("/content/drive/MyDrive/output/Combine_IoC_13_11_2022.csv",index=False)
df2 = pd.DataFrame(IOC_Final_list)
df2.to_csv("/content/drive/MyDrive/output/Final_IOC_13_11_2022.csv",index=False)
print("Success")

Success


In [ ]:
df2 = pd.DataFrame(IOC_Final_list)
len(df2)

195514

In [ ]:
df2

,tweet_date,account,ioc_type,ioc_value,type_of_attack,tweet_id,text,Mentioned Hashtag,Expanded URL
0,2021-01-01 00:09:03+00:00,MalwarePatrol,url,http://dubioustimes.com/,[phishing],1.344798e+18,Another active #Phishing targeting Google Driv...,"['Phishing', 'onpatrol4malware']",['https://twitter.com/MalwarePatrol/status/134...
1,2021-01-01 00:09:03+00:00,MalwarePatrol,url,https://t.co/m4mjDMt7az,[phishing],1.344798e+18,Another active #Phishing targeting Google Driv...,"['Phishing', 'onpatrol4malware']",['https://twitter.com/MalwarePatrol/status/134...
2,2021-01-01 00:09:03+00:00,MalwarePatrol,domain,dubioustimes.com,[phishing],1.344798e+18,Another active #Phishing targeting Google Driv...,"['Phishing', 'onpatrol4malware']",['https://twitter.com/MalwarePatrol/status/134...
3,2021-01-01 00:09:03+00:00,MalwarePatrol,domain,t.co,[phishing],1.344798e+18,Another active #Phishing targeting Google Driv...,"['Phishing', 'onpatrol4malware']",['https://twitter.com/MalwarePatrol/status/134...
4,2021-01-01 00:26:26+00:00,PhishStats,url,http://departmen-advant-page.my.id/,[phishing],1.344802e+18,new #phishing at hXXp://departmen-advant-page[...,"['phishing', 'infosec', 'cybersecurity']",['https://twitter.com/PhishStats/status/134480...
...,...,...,...,...,...,...,...,...,...
195509,2021-07-31 23:06:42+00:00,BethkazV,url,https://www.esetphishingderby.com?score=65,[phishing],1.421608e+18,Test your knowledge at https://t.co/yvg8zciMOZ...,"['eset', 'antivirus', 'phishing', 'contest', '...",['https://www.esetphishingderby.com?score=65']
195510,2021-07-31 23:06:42+00:00,BethkazV,domain,t.co,[phishing],1.421608e+18,Test your knowledge at https://t.co/yvg8zciMOZ...,"['eset', 'antivirus', 'phishing', 'contest', '...",['https://www.esetphishingderby.com?score=65']
195511,2021-07-31 23:37:40+00:00,ecarlesi,domain,t.co,"[opendir, phishing]",1.421616e+18,Threat on hxxp://certiport[.]games/postale[.]z...,"['phishing', 'opendir', 'namedotcom']",['https://twitter.com/ecarlesi/status/14216160...
195512,2021-07-31 23:50:02+00:00,HeliosCert,ip,185.27.116.4,[malware],1.421619e+18,Sample submitted\n2021-07-31 23:50:02\nDionaea...,"['malware', 'cyber', 'security']",NaN


In [ ]:
list_url=['https://www.facebook.com/zombieslayerfans','https://youtu.be/TkgcChi6fhQ','https://bit.ly/2EJ90Ih', 'https://bit.ly/3itE0du', 'https://twitter.com/i/events/1417537865446150146', 'https://twitter.com/B2Spirit_TT/status/1417640231839453184/photo/1']
full_url=list_url.copy()
print(full_url)
for i in range(0,len(full_url)):
  x=re.search(twitter_expression,full_url[i])
  y=re.search(youtube_expression,full_url[i])
  z=re.search(facebook_expression,full_url[i])
  print(x)
  print(y)
  print(z)
  if(re.search(twitter_expression,full_url[i]) or re.search(youtube_expression,full_url[i]) or re.search(facebook_expression,full_url[i])):
    print("T")
    list_url.remove(full_url[i])
print(list_url)

['https://www.facebook.com/zombieslayerfans', 'https://youtu.be/TkgcChi6fhQ', 'https://bit.ly/2EJ90Ih', 'https://bit.ly/3itE0du', 'https://twitter.com/i/events/1417537865446150146', 'https://twitter.com/B2Spirit_TT/status/1417640231839453184/photo/1']
None
None
<re.Match object; span=(0, 41), match='https://www.facebook.com/zombieslayerfans'>
T
None
<re.Match object; span=(0, 28), match='https://youtu.be/TkgcChi6fhQ'>
None
T
None
None
None
None
None
None
<re.Match object; span=(8, 48), match='twitter.com/i/events/1417537865446150146'>
None
None
T
<re.Match object; span=(8, 66), match='twitter.com/B2Spirit_TT/status/141764023183945318>
None
None
T
['https://bit.ly/2EJ90Ih', 'https://bit.ly/3itE0du']


In [ ]:
len(tweets)

110877

In [ ]:
'''		'''

'\t\t'

In [ ]:
ip="02:31:45"
ip_split=str(ip).split(":")
print(ip_split)
print(len(ip_split))
if(len(ip_split)>=2):
	print("sucess")

['02', '31', '45']
3
sucess


In [ ]:
ip=['116.85.15.146', '06:19:26', '107.172.248.177']
date_list=[]
for loc in range(0,len(ip)):
  ip_split=str(ip[loc]).split(":")
  print(ip_split)
  print(len(ip_split))
  if(len(ip_split)>=2):
	  date_list.append(ip[loc])
  print(date_list)
for dateloc in range(0,len(date_list)):
  ip.remove(date_list[dateloc])   
print(ip)

['116.85.15.146']
1
[]
['06', '19', '26']
3
['06:19:26']
['107.172.248.177']
1
['06:19:26']
['116.85.15.146', '107.172.248.177']


In [ ]:
a=['cybersecurity', 'phishing', 'malware', 'Infosec', 'cyberthreats', 'ramsomware', 'hacking', 'databreach', 'dataprotection', 'privacy', 'dataleak', 'cyberattacks']
b=['malware','ransomware','botnet','androidtrojan','bankingtrojan','windowstrojan','remoteaccesstrojan','rattrojan','linuxtrojan','adware','spyware','keylogger','rootkit','bootkit','rigek','exploitkit','honeypot','ioc','malspam','crypto-locker','cryptolocker','crypto-ransomware','opendir','c2','md5','sha256','spearphishing','spear-phishing','phishing','phishingkit','cyberattack','cve','rootkit','sqlinjection','keylogging','0day','keylogger','passwordcracker','metasploit','reduc','zombie','C&C']
c=set(a) & set(b)
c

{'malware', 'phishing'}